Hi Mike, this is a markdown cell

you need to install the Jupyter from microsoft extension in Visual Studio Code (easiest) from the left panel - plugins  


In [2]:
1+2+3+4

10

In [146]:
# TODO: import needed modules

from lxml import etree
import math as m
import numpy as np
import pandas as pd
import attr
import collections 
import pathlib as pl
import re
# for colored text
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


File Directories can be easily copied with the following procedure:
1) open windows files manager and go to directory
2) on the directory press and hold the shift key and left click the mouse and select - "Copy as path"
3) key press "control v" below

In [18]:
# TODO: input QC dir
inputDir=input("Input Dir Path to Be Checked")
inputDir=pl.Path(inputDir)
# print(inputDir)
inputDir

WindowsPath('C:/Users/arond/OneDrive/EA local files to upload when done/Working files/5EADD00001 - Randall, Elenor - 1821 Knutsford Place')

In [35]:
# TODO: check dir for h2k files
def get_files(extensions):
    all_files = {}
    for ext in extensions:
        
        all_files[ext]=[{"fileName":f.name,'file':f} for f in (inputDir.glob(ext))]
        # all_files.extend(pl.Path('.').glob(ext))
    return all_files

files = get_files(('*.h2k', '*.jpg', '*.pdf'))
files
# TODO: warn of extra h2k files

# TODO: warn of missing required photos

# TODO: warn of missing ERS output files

{'*.h2k': [{'fileName': '5EADD00001.h2k',
   'file': WindowsPath('C:/Users/arond/OneDrive/EA local files to upload when done/Working files/5EADD00001 - Randall, Elenor - 1821 Knutsford Place/5EADD00001.h2k')},
  {'fileName': '5EXXD0003.h2k',
   'file': WindowsPath('C:/Users/arond/OneDrive/EA local files to upload when done/Working files/5EADD00001 - Randall, Elenor - 1821 Knutsford Place/5EXXD0003.h2k')}],
 '*.jpg': [{'fileName': '5EADD00001 - Datasheet.jpg',
   'file': WindowsPath('C:/Users/arond/OneDrive/EA local files to upload when done/Working files/5EADD00001 - Randall, Elenor - 1821 Knutsford Place/5EADD00001 - Datasheet.jpg')},
  {'fileName': '5EXXD0003-ATTIC ACCESS.jpg',
   'file': WindowsPath('C:/Users/arond/OneDrive/EA local files to upload when done/Working files/5EADD00001 - Randall, Elenor - 1821 Knutsford Place/5EXXD0003-ATTIC ACCESS.jpg')},
  {'fileName': '5EXXD0003-BACK.jpg',
   'file': WindowsPath('C:/Users/arond/OneDrive/EA local files to upload when done/Working fil

WindowsPath('C:/Users/arond/OneDrive/EA local files to upload when done/Working files/5EADD00001 - Randall, Elenor - 1821 Knutsford Place/5EADD00001.h2k')

In [39]:
# TODO: read h2k xml file
# https://lxml.de/tutorial.html
from lxml import etree
try:
  from lxml import etree
  print("running with lxml.etree")
except ImportError:
  try:
    # Python 2.5
    import xml.etree.cElementTree as etree
    print("running with cElementTree on Python 2.5+")
  except ImportError:
    try:
      # Python 2.5
      import xml.etree.ElementTree as etree
      print("running with ElementTree on Python 2.5+")
    except ImportError:
      try:
        # normal cElementTree install
        import cElementTree as etree
        print("running with cElementTree")
      except ImportError:
        try:
          # normal ElementTree install
          import elementtree.ElementTree as etree
          print("running with ElementTree")
        except ImportError:
          print("Failed to import ElementTree from any known place")

# https://stackoverflow.com/questions/42896807/how-to-parse-xml-from-local-file-or-url-with-lxml
# https://lxml.de/parsing.html
tree = etree.parse(str(files['*.h2k'][0]['file']))
tree


running with lxml.etree


In [155]:
p=r"/".join([r'./','Codes','Window','Standard','Code'])
# p=r"/".join([r'./','Codes','Window','Standard'])

p=".//ProgramInformation"
print (p)
t=tree.xpath(p)
t=t[0]
tl=t.getchildren()
# [ (i.attrib,j.attrib, z.tag,z.attrib, y.text)  for i in tl for j in i.getchildren() for z in j.getchildren() for y in z.getchildren()]
# [(i.getparent().getparent().tag,i.getparent().tag,i.getparent().getparent().tag) for i in tree.xpath(p)]
# [(i.getparent().tag,i.attrib, i.tag, i.text) for i in tree.xpath(p)]
l=[i.tag for  j in tl for i in list(j.getiterator())]

weather=p+"/".join(['/Weather', 'Region','English'])
fileInfo=p+"/".join(['/File'])
weather=tree.xpath(p+"/".join(['/Weather', 'Region','English']))[0].text,tree.xpath(p+"/".join(['/Weather', 'Location','English']))[0].text
print(f"Weather Info:{weather}")
fileInfo=tree.xpath(p+"/".join(['/File']))[0].attrib,tree.xpath(p+"/".join(['/File', 'Identification']))[0].text
print(f"File Info:{fileInfo}")
tags=[i.tag for i in tree.xpath(p+"/".join(['/File']))[0]]
tree.xpath(p+'/File')
# [j for i in tree.xpath(p+'/File/'+tags[0]) for j in i]


# TODO: display basic File data


for f in tree.xpath(p+'/File'): 
  # for j in i]
  print (f"Program Info :")
  for t in f:
    if t.tag=='Ownership':
      print(f"   {t.tag}: {t.find('English').text}")
    else:
      print(f"   {t.tag}: {t.text}")


# TODO: display basic customer data

for f in tree.xpath(p+'/Client'): 
  print (f"Client Info :")
  for t in f:
    if t.tag=='Name':
      print(f"   {t.tag}: First :{t.find('First').text}, Last : {t.find('Last').text} ")
    if t.tag=='Telephone':
      # ensure 9 numbers
      if len(re.findall('[0-9]',t.text) )!= 9:
        print(f"   {bcolors.WARNING}****************** Phone number is incorrect:  *****************************{bcolors.ENDC}")
      print(f"   {t.tag}: {t.text}")
    else:
      print(f"   {t.tag}: {t.text}")


# client=p+"/".join(['/File', 'Region','English'])
# [i.text for i in tree.xpath('.//Weather') for j in i.]
# subTags=[i.tag for i in tree.xpath(p)[0]]
# tree.xpath(p+'/'+subTags[0])
#  [i.tag for i in tree.xpath(p+subTags)[0][0]]

# [i for i in subTags]
# l


.//ProgramInformation
Weather Info:('BRITISH COLUMBIA', 'ABBOTSFORD')
File Info:({'evaluationDate': '2021-10-11'}, 'Chris ')
Program Info :
   Identification: Chris 
   PreviousFileId: None
   EnrollmentId: None
   Ownership: Dwelling private
   TaxNumber: 01-332-00657-000
   EnteredBy: Aron Dalton
   UserTelephone: 250-221-1127
   UserExtension: None
   CompanyTelephone: None
   CompanyExtension: None
   Company: Ridge Energy
   BuilderName: None
   HomeownerAuthorizationId: None
Client Info :
   Name: First :Chris , Last : Waller 
   Name: 
                
   ****************** Phone number is incorrect:  *****************************
   Telephone: (604) 723-51
   StreetAddress: 
                
   MailingAddress: 
                


In [ ]:
# TODO: check files in  directory names for extra spaces an jpg extentions...

In [2]:
# TODO: check all reqired files are uploaded